In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

In [ ]:
import os

from tqdm import tqdm
import numpy as np
import pandas as pd

import anndata
import scanpy as sc

import torch

from scmg.preprocessing.data_standardization import GeneNameMapper
from scmg.model.contrastive_embedding import CellEmbedder, embed_adata, decode_adata

gene_name_mapper = GeneNameMapper()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.autolayout"] = False
matplotlib.rc('pdf', fonttype=42)
plt.rcParams['font.family'] = 'FreeSans'
sc.set_figure_params(vector_friendly=True, dpi_save=300)
plt.rcParams['axes.grid'] = False

In [ ]:
output_path = 'Travaglini_Lung_cell_type_search_plots'
os.makedirs(output_path, exist_ok=True)

In [ ]:
# Load the autoencoder model
model_ce_path = '../../../contrastive_embedding/trained_embedder/'

model = torch.load(os.path.join(model_ce_path, 'model.pt'))
model.load_state_dict(torch.load(os.path.join(model_ce_path, 'best_state_dict.pth')))

device = 'cuda:0'
model.to(device)
model.eval()

In [ ]:
adata_ct_ref = sc.read_h5ad('../../ref_cell_adata.h5ad')
adata_ct_ref

In [ ]:
#adata = sc.read_h5ad('../../../contrastive_embedding/test_embedding/Burclaff_intestine_HS_2022_all.h5ad')
adata = sc.read_h5ad('../../../contrastive_embedding/test_embedding/Travaglini_Lung_HS_2021_10x.h5ad')

adata.var.index = adata.var['feature_id']
adata

In [ ]:
embed_adata(model, adata, batch_size=8192)

In [ ]:
major_ct_df = pd.read_csv('../../../cell_type_analysis/major_cell_type_annotation.csv')
major_ct_df

In [ ]:
sorted(major_ct_df['major_cell_type'].value_counts().index)

In [ ]:
from scmg.model.cell_type_search import CellTypeSearcher

cts = CellTypeSearcher(adata_ct_ref)

In [ ]:
adata.obs['cell_type'].value_counts()

In [ ]:
#query_emb = adata.obsm['X_ce_latent'][adata.obs['cell_type'] == 
#                        'colon goblet cell']
query_emb = adata.obsm['X_ce_latent']

cell_match_df = cts.search_ref_cell(query_emb)

adata.obsm['X_project_umap'] = cell_match_df[['umap_x', 'umap_y']].values
adata.obs['project_dist'] = cell_match_df['distance'].values

In [ ]:
sc.pl.umap(adata_ct_ref, color='major_cell_type')

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=300)
sc.pl.umap(adata_ct_ref, ax=ax, show=False)
sc.pl.embedding(adata, basis='X_project_umap', color='cell_type', ax=ax, frameon=False)
#sc.pl.embedding(adata, basis='X_project_umap', color='project_dist', ax=ax)

fig.savefig(os.path.join(output_path, 'umap_projected_cells.pdf'))

In [ ]:
plt.scatter(adata_ct_ref.obsm['X_umap'][:, 0], adata_ct_ref.obsm['X_umap'][:, 1], 
            s=1, c='lightgrey')
#plt.scatter(cell_match_df['umap_x'], cell_match_df['umap_y'], s=1, c='red')
sct = plt.scatter(cell_match_df['umap_x'], cell_match_df['umap_y'],
             s=1, c=cell_match_df['distance'], cmap='viridis')
plt.colorbar(sct)

In [ ]:
query_cell_types = np.unique(adata.obs['cell_type'])

all_ct_match_df = pd.DataFrame(
    data=np.zeros((len(query_cell_types), len(cts.cell_types)), dtype=np.float32),
    index=query_cell_types, columns=cts.cell_types
)

for ct in tqdm(query_cell_types):
    query_emb = adata.obsm['X_ce_latent'][adata.obs['cell_type'] == ct]
    ct_match_df = cts.search_ref_cell_types(query_emb)

    all_ct_match_df.loc[ct] = ct_match_df['weight']
    

In [ ]:
for ct in query_cell_types:
    display(all_ct_match_df.loc[ct].sort_values(ascending=False)[:5])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
sc.pl.embedding(adata, basis='X_project_umap', color='cell_type',
                legend_loc='on data', ax=ax, legend_fontsize=5)

In [ ]:
query_cts_to_show = [
    'lung goblet cell', 
    'mucus secreting cell', 
    'respiratory basal cell', 
    'basal cell', 
    'club cell', 
    'type I pneumocyte', 
    'type II pneumocyte', 
    'epithelial cell', 
    'pulmonary ionocyte', 
    'tracheobronchial serous cell', ]

ref_cts_to_show = [
    'tracheobronchial goblet cell', 
    'respiratory basal cell', 
    'lung secretory cell', 
    'epithelial cell of alveolus of lung', 
    'type I pneumocyte', 
    'type II pneumocyte', 
    'pulmonary ionocyte', 
    'serous cell of epithelium of bronchus',
    ]


ct_mtx_to_show = all_ct_match_df.loc[query_cts_to_show, ref_cts_to_show]
#ct_mtx_to_show = ct_mtx_to_show / ct_mtx_to_show.values.max(axis=1)[:, None]

fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(ct_mtx_to_show.T, cmap='viridis', ax=ax, vmax=0.4, cbar_kws={'label': 'cell type match score'})
ax.set_xlabel('query cell type')
ax.set_ylabel('reference cell type')

fig.savefig(os.path.join(output_path, 'cell_type_match_heatmap.pdf'))

In [ ]:
adata.obs['project_dist'] = cell_match_df['distance'].values
adata.obs['ref_cell'] = cell_match_df['ref_cell'].values
adata.obs['projected_cell_type'] = adata_ct_ref.obs['cell_type'].loc[
                                            cell_match_df['ref_cell']].values

projected_cell_count_map = adata.obs['ref_cell'].value_counts().to_dict()
adata_ct_ref.obs['projected_cell_count'] = adata_ct_ref.obs.index.map(
    lambda x: projected_cell_count_map.get(x, 0))

ct_confusion_df = pd.crosstab(adata.obs['cell_type'], adata.obs['projected_cell_type'])
ct_confusion_df = ct_confusion_df / ct_confusion_df.values.sum(axis=1)[:, None]

for ct_query in ct_confusion_df.index:
    display(ct_confusion_df.loc[ct_query].sort_values(ascending=False).head(5))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 12))


sns.heatmap(ct_confusion_df.loc[
    [
    'plasma cell',
    'B cell', 

    'natural killer cell',
    'lymphocyte',
    'mature NK T cell',
    
    'effector memory CD4-positive, alpha-beta T cell',
    'effector memory CD8-positive, alpha-beta T cell',
    'naive thymus-derived CD4-positive, alpha-beta T cell',
    'naive thymus-derived CD8-positive, alpha-beta T cell',

    
    'plasmacytoid dendritic cell, human',
    'CD1c-positive myeloid dendritic cell',
    'myeloid dendritic cell, human',
    'myeloid leukocyte',
    'monocyte',
    'classical monocyte',
    'intermediate monocyte',
    'non-classical monocyte',
    'dendritic cell',
    'macrophage',
    'alveolar macrophage',
    
    'megakaryocyte',

    'endothelial cell of artery',
    'endothelial cell',
    'capillary endothelial cell',
    'vein endothelial cell',
    'endothelial cell of lymphatic vessel',
    'pericyte',

    'vascular associated smooth muscle cell',
    'bronchial smooth muscle cell',
    'myofibroblast cell',
    'pulmonary interstitial fibroblast',
    'fibroblast',
    
    'mesothelial cell of pleura',
    
    'mucus secreting cell',
    'club cell',
    'basal cell',
    'respiratory basal cell',
    'ciliated cell',
    'lung ciliated cell',
    'lung goblet cell',
    'tracheobronchial serous cell',
    'pulmonary ionocyte',
    'type I pneumocyte',
    'type II pneumocyte',
    'epithelial cell',

    'lung neuroendocrine cell',
    ], 
    [
    'plasma cell',
    'B cell', 

    'CD16-positive, CD56-dim natural killer cell, human',
    'natural killer cell',

    'T cell',

    'plasmacytoid dendritic cell',
    'conventional dendritic cell',
    'mast cell',
    'classical monocyte',
    'monocyte',
    'non-classical monocyte',
    'macrophage',
    'alveolar macrophage',

    'megakaryocyte',

    'pulmonary artery endothelial cell',
    'capillary endothelial cell',
    'vein endothelial cell',
    'endothelial cell of lymphatic vessel',
    'pericyte',

    'tracheobronchial smooth muscle cell',
    'vascular associated smooth muscle cell',
    'myofibroblast cell',
    'fibroblast',
    'alveolar type 1 fibroblast cell',
    'mesothelial cell',

    'epithelial cell of lower respiratory tract',
    'epithelial cell of alveolus of lung',
    'respiratory basal cell',
    'lung ciliated cell',
    'ciliated columnar cell of tracheobronchial tree',
    'nasal mucosa goblet cell',
    'tracheobronchial serous cell',
    'ionocyte',
    'type I pneumocyte',
    'type II pneumocyte',

    'enteroendocrine cell',   
    ]].T,
    cmap='viridis', vmax=0.7)

fig.savefig(os.path.join(output_path, 'cell_type_projection_confusion_matrix.pdf'))